In [1]:
!pip install chromadb
!pip install openai
!pip install tqdm
!pip install sounddevice
!pip install whisper
!pip install gtts
!pip install beautifulsoup4
!pip install tavily-python
!pip install lxml
!pip install pyttsx3
!pip install langchain_openai
!pip install langchain-community
!pip install langchain-google-genai
!pip install langsmith
!pip install google-search-results

!pip install lxml
!pip install httpx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 115.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 131.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.9/131.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 9.8 MB/s 

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=6962e1d69cbc5cbe48c40128882f5cfa74fc1c3e3dd75cd2f451450d35c43c56
  Stored in directory: /root/.cache/pip/wheels/0c/47/f5/89b7e770ab2996baf8c910e7353d6391e373075a0ac213519e
Successfully built google-search-results


In [2]:
# ============================================
# STEP - 1 : Imports
# ============================================

# Standard Library Imports
import os
import json
import re

# Third-Party Library Imports
import chromadb
import openai
import tqdm
import pyttsx3
import gtts
from google.colab import drive
from bs4 import BeautifulSoup
from tavily import TavilyClient

# LangChain and related framework imports
import langchain
from langchain import hub
from langchain.tools import Tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import WebBaseLoader
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser # Corrected import path
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

In [3]:
# =========================================
# STEP - 2 : Load API keys (Colab + Env)
# =========================================
import os

# Attempt to fetch from Colab userdata if available
try:
    from google.colab import userdata
    _colab_available = True
except ImportError:
    _colab_available = False

def get_key(key_name: str) -> str:
    """Fetch API key from Colab userdata first, then environment variables."""
    key = None
    if _colab_available:
        key = userdata.get(key_name)
    if not key:
        key = os.environ.get(key_name)
    if not key:
        print(f"❌ {key_name} not found! Please add it to Colab secrets or environment variables.")
    else:
        print(f"✅ {key_name} loaded successfully!")
    return key

# Fetch all keys
OPENAI_API_KEY = get_key("OPENAI_API_KEY")
TAVILY_API_KEY = get_key("TAVILY_API_KEY")
SERPAPI_API_KEY = get_key("SERPAPI_API_KEY")
LANGCHAIN_API_KEY = get_key("LANGCHAIN_API_KEY")
# LANGSMITH_API_KEY = get_key("LANGSMITH_API_KEY")
# HF_TOKEN = get_key("HF_TOKEN")
# Optional: Pinecone if you use it
# PINECONE_API_KEY = get_key("PINECONE_API_KEY")

# Set OpenAI key for SDKs
if OPENAI_API_KEY:
    import openai
    openai.api_key = OPENAI_API_KEY


✅ OPENAI_API_KEY loaded successfully!
✅ TAVILY_API_KEY loaded successfully!
✅ SERPAPI_API_KEY loaded successfully!
✅ LANGCHAIN_API_KEY loaded successfully!


In [4]:
# ================================================
# ---------- MOUNT THE GOOGLE DRIVE -------------
# ================================================

from google.colab import drive
drive.mount('/content/drive')

print(f"✅ Google Drive mounted successfully!")


Mounted at /content/drive
✅ Google Drive mounted successfully!


In [5]:
# ============================================
# STEP - 3 : Importing all embedded chunks
# ============================================

all_embedded_file_path = "/content/drive/MyDrive/Ironhack_final_project/all_embedded_chunks.json"

with open(all_embedded_file_path, "r", encoding="utf-8") as f:
    all_embedded_chunks = json.load(f)

print(f"✅ Loaded {len(all_embedded_chunks)} chunks")
print(all_embedded_chunks[0])  # preview first chunk

✅ Loaded 164 chunks
{'video_id': 'tQ84XYcP-nA', 'chunk_index': 0, 'text': "Every week there's a new AI tool making headlines, and right now there are more AI video generators than ever. But most of them don't work as well as you'd expect. Some generate great videos, but only if you stay within their style limits. If you try to get more creative, like detailed anime or wild fantasy worlds, they often mess up, and most of the time they're too expensive or just confusing to use. I've tested every major tool that's come out recently, and I've seen where they shine and where they completely fall apart. So, in this video, I'm going to show you the AI video tools that actually deliver the kind of quality you'd want to publish. The best way to use them without wasting hours learning clunky software, and the one platform that ties everything together, so you can create full videos without switching between five different sites. Let's break it down. All right, let's start with Seedance 1.0. Sean

In [9]:
# ============================================
# STEP - 4 : RAG Pipeline with Memory
# ============================================

import os
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory

# --------------------------
# 1️ Initialize OpenAI client
# --------------------------
# OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")  # Load API key from Colab secrets
# client = OpenAI(api_key=api_key) # No need to re-initialize client here


# --------------------------
# 2️ Initialize OpenAIEmbeddings
# --------------------------
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENAI_API_KEY
)

# --------------------------
# 3️ Create/Load vector store
# --------------------------

persist_dir = "/content/drive/MyDrive/Ironhack_final_project/chromadb_store" # file path to the vector database



vectorstore = Chroma(
    persist_directory=persist_dir,  # folder where your vector DB is stored
    embedding_function=embeddings
)


# ----------------------------------------------------------
# 4️ Add embedded chunked data to Chromadb_store (vector DB or store)
# ----------------------------------------------------------
texts = [chunk["text"] for chunk in all_embedded_chunks]
metadatas = [
    {
        "source": chunk.get("video_id") or chunk.get("article_id"),
        "chunk_index": chunk["chunk_index"]
    }
    for chunk in all_embedded_chunks
]

# Add documents with automatic embedding generation
vectorstore.add_texts(texts=texts, metadatas=metadatas)

print(f"🎉 Done! Chroma vector store is loaded")


# --------------------------
# 5️ Create retriever
# --------------------------
retriever = vectorstore.as_retriever(search_kwargs={"k": 8})  # higher k for better results


# --------------------------
# 6️ LLMChain prompt and chain
# --------------------------

prompt = PromptTemplate(
    input_variables=["context", "question", "chat_history"],
    template="""
You are a helpful AI assistant for content creators.
Always answer concisely, clearly, and in a structured format.
Prefer bullet points or numbered steps (3–6 items).
Each bullet should be 1–2 sentences max.
Do not repeat information.
If the context is incomplete, use conversation history or your own knowledge.

Conversation history:
{chat_history}

Context:
{context}

Question: {question}

Answer (concise and structured):
"""
)

# prompt = PromptTemplate(
#     input_variables=["context", "question", "chat_history"],
#     template="""
# You are a helpful AI assistant. Use the context below and the chat history to answer the question.
# If the context is incomplete, rely on conversation history or your knowledge to give the best answer.

# Conversation history:
# {chat_history}

# Context:
# {context}

# Question: {question}

# Answer:
# """
# )


# -------------------------------------------------------------------------------------------------------

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3, openai_api_key=api_key) # Pass the api_key here

# -------------------------------------------------------------------------------------------------------


# ----------------------
# Add memory here
# ----------------------
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


qa_chain = LLMChain(llm=llm, prompt=prompt)


# ----------------------------------
#  Full RAG function with memory
# ----------------------------------

def ask_rag(query: str) -> str:
    """
    Full RAG pipeline: retrieves relevant chunks, uses memory, and generates answer.
    """
    # --- Retrieve relevant chunks ---
    results = retriever.get_relevant_documents(query)

    if not results:
        return "❌ No relevant documents found."

    # --- Combine retrieved chunks ---
    context = "\n\n".join([doc.page_content for doc in results])

    print("\n📝 Retrieved context preview:\n", context[:1000])

    # --- Load memory ---
    chat_history = memory.load_memory_variables({}).get("chat_history", [])

    # --- Generate answer ---
    answer = qa_chain.run({
        "context": context,
        "question": query,
        "chat_history": chat_history
    })

    # --- Save interaction into memory ---
    memory.save_context({"input": query}, {"output": answer})

    return answer

🎉 Done! Chroma vector store is loaded


In [7]:
# ============================================
# STEP - 5 :  Testing my RAG retrieval and its memory with multiple questions
# ============================================

# First question: sets some context
print(ask_rag("What is an AI image generator?"))

# Second question: relies on memory
print(ask_rag("And which ones are considered the best?"))

# Third question: continues from memory
print(ask_rag("Pick one for me"))


# # -----------------------
# # 8️⃣ Testing my RAG
# # -----------------------
# if __name__ == "__main__":
#     user_query = input("❓ Enter your question: ")
#     answer = ask_rag(user_query)
#     print("\n💡 Answer:\n", answer)


# # -----------------------------------------------------------
# # 8️⃣ Testing RAG with memory with continuous questioning
# # -----------------------------------------------------------

# if __name__ == "__main__":
#     print("📝 You can ask multiple questions. Type 'exit' to quit.\n")

#     while True:
#         user_query = input("❓ Enter your question: ")
#         if user_query.lower() in ["exit", "quit"]:
#             print("👋 Exiting chat. Goodbye!")
#             break

#         answer = ask_rag(user_query)
#         print("\n💡 Answer:\n", answer)
#         print("-" * 50)  # separator for readability



/tmp/ipython-input-1001266257.py:137: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)



📝 Retrieved context preview:
 AI has revolutionized image creation. You can now type in a text prompt, like design a storefront for a cookie store, and get captivating visuals back. There are lots of different AI image generators out there, both free and also paid, and you've probably used some yourself. In this video, in partnership with Zapier, we'll look at the top seven AI image generators from Dalle 3 to Midjourney and more. Let's dive in. First up, we have Dalle 3, which is developed by OpenAI, the maker of ChatGPT. You can access this directly from chatgpt.com with a free account, although you only get two images per day. With a plus account, on the other hand, you can create more images, but plans start at $20 per month. As an alternative, you can go to Microsoft's Bing Image Creator, which allows you to create any number of images for free, and it uses Dalle 3 on the backend, the same as what you get with ChatGPT. So, what do I like or dislike about it? Well, overall, you get

/tmp/ipython-input-1001266257.py:151: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run({


- **Definition**: An AI image generator is a software tool that creates images based on textual prompts provided by the user.
- **Functionality**: Users can input descriptions, and the AI interprets these to produce visuals that match the given criteria.
- **Technology**: These generators utilize advanced machine learning models, often trained on vast datasets of images and text.
- **Applications**: They are used in various fields, including art, marketing, design, and entertainment, to create unique visuals quickly.
- **Examples**: Popular AI image generators include Dalle 3, Midjourney, and Stable Diffusion.

📝 Retrieved context preview:
 pretty good, but what do I not like? It's very resource intensive, so you'll need a powerful computer to be able to run it, which could be a limitation. Last up at number seven, we have Generative AI by Getty Images, which was developed in partnership with NVIDIA. This is by far the most expensive of all the offerings, starting at $49 for just 25 im

**==================>>> T O O L S <<<========================**

In [24]:
# ============================================
# STEP - 6 : Retriever tool (Chroma retriever) with metadata
# ============================================

from langchain.tools import Tool

retriever_tool = Tool(
    name="Chroma Retriever",
    func=vectorstore.as_retriever().get_relevant_documents,
    description=(
        "This is the PRIMARY tool to use first. "
        "Always try this before using any other tool. "
        "It retrieves relevant document chunks from the Chroma database "
        "to answer questions when the context exists."
    )
)

print("✅ Retriever tool is loaded")

✅ Retriever tool is loaded


In [12]:
# ============================================
# STEP - 7 : Google Search tool (Tavily)
# ============================================


# from langchain.tools import Tool


# TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')
# if not TAVILY_API_KEY:
#     raise ValueError("❌ No Tavily API key found! Please add it in Colab secrets.")

from tavily import TavilyClient
tavily = TavilyClient(api_key=TAVILY_API_KEY)

def search_tavily(query: str):
    """Perform a web search using Tavily and return top results as a list."""
    results = tavily.search(query, max_results=3)
    # Return a list of result content
    return [r["content"] for r in results["results"]]

search_tool = Tool(
    name="Google Search (Tavily)",
    func=search_tavily,
    description="Use this when the question cannot be answered from the context. Returns top 3 web search results."
)

print("✅ Search tool is loaded")

# ------------------------------------
# Testing the search tool (Tavily)
# ------------------------------------

results = search_tavily("AI tools for writing blogs")
print(f"🔍 Tavily Search Results:")
for i, r in enumerate(results, 1):
    print(f" {i}. {r}")

✅ Search tool is loaded
🔍 Tavily Search Results:
 1. My top 7 picks for the best AI for writing SEO rich blog content tools –. Koala AI. WriteHuman. Copymatic. Jenni AI.
 2. With RightBlogger, you can write SEO-friendly articles in minutes, turn any videos & audio recordings into blog posts, and create interactive AI lead magnets to grow your email list—all using simple, powerful AI tools. | Feature | RightBlogger | Other AI Tools | ChatGPT | | AI Blogging Tools |  |  |  | RightBlogger Tool Studio lets you create custom AI-powered tools for your website that engage readers, capture leads, and drive traffic. That said, we do believe it's important for bloggers and content creators to use these AI tools responsibly. Powerful AI tools like the 80+ inside RightBlogger are designed to give you a great starting point to all of your content creation needs. Can AI-generated content be detected by tools?
 3. Go from blank page to polished blog post in seconds with Grammarly, your AI writing par

In [16]:
# ========================================================
# STEP 8 - W H I S P E R  T O O L - for Speech-to-Text
# ========================================================

# import whisper # Removed this import
# import sounddevice as sd # Keep this import if local recording is desired, but address PortAudio issue separately
from scipy.io.wavfile import write
from langchain.tools import Tool
from openai import OpenAI # Ensure OpenAI client is imported

# Load Whisper model once # Removed this line



client = OpenAI(api_key=OPENAI_API_KEY)


def record_audio(filename="input.wav", duration=5, fs=16000):
    print(f"🎤 Recording for {duration} seconds...")
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=1)
    sd.wait()
    write(filename, fs, audio)
    print("✅ Recording finished")
    return filename

def transcribe_audio_openai(audio_file_path):
    """Transcribes audio using OpenAI's Whisper model."""
    with open(audio_file_path, "rb") as audio_file:
        # Use the existing 'client' object from cell oz4C828MYuR5
        # Make sure the client is initialized in a preceding cell
        transcription = client.audio.transcriptions.create(
            model="whisper-1", # OpenAI's Whisper model
            file=audio_file
        )
    return transcription.text

# --- Wrap in a Tool ---
def whisper_speech_to_text(audio_file_path):
    """Transcribes a given audio file into text using OpenAI's Whisper."""
    # Note: This now expects a file path as input, not live recording due to PortAudio issue
    # If local recording is resolved, you can add logic here to record first
    if not os.path.exists(audio_file_path):
         return f"❌ Error: Audio file not found at {audio_file_path}"

    text = transcribe_audio_openai(audio_file_path)
    return text

whisper_tool = Tool(
    name="Whisper Speech-to-Text",
    func=whisper_speech_to_text,
    description=(
        "Transcribes an audio file (provide file path) into text using OpenAI's Whisper model. "
        # Removed the part about recording live audio due to environment limitations
        "Useful for converting spoken content from a file into text for analysis or response generation."
    )
)

print("✅ Whisper tool is loaded (using OpenAI)")

✅ Whisper tool is loaded (using OpenAI)


In [17]:
# =====================================================
# STEP - 9 : TTS (Text-to-Speech) tool with OpenAI
# =====================================================

from openai import OpenAI
import os


# client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def text_to_speech(text, filename="response.mp3"):
    """
    Convert agent's text response to natural speech using OpenAI TTS
    """
    with client.audio.speech.with_streaming_response.create(
        model="gpt-4o-mini-tts",   # You can also try "gpt-4o-tts" for higher quality
        voice="alloy",             # Options: alloy, verse, sage, etc.
        input=text,
    ) as response:
        response.stream_to_file(filename)

    print(f"🔊 Saved speech to {filename}")
    return filename

# Wrap text-to-speech function as a Tool
tts_tool = Tool(
    name="Text-to-Speech",
    func=text_to_speech,
    description="Converts text into natural-sounding speech using OpenAI TTS and saves as MP3."
)

print("✅ Text-to-Speech tool is loaded")

✅ Text-to-Speech tool is loaded


In [18]:

# =================================================
# STEP - 10 : N E W S  T O O L - Powered by Google
# =================================================

import os
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool

# --------------------------
# 1️⃣ Load SerpAPI key
# --------------------------
# For Colab, you can use userdata.get() if available
try:
    from google.colab import userdata
    serpapi_api_key = userdata.get('SERPAPI_API_KEY')
except ImportError:
    serpapi_api_key = os.getenv('SERPAPI_API_KEY')

if not serpapi_api_key:
    raise ValueError("❌ No SerpAPI API key found! Please set it in environment variables or Colab secrets.")

os.environ['SERPAPI_API_KEY'] = serpapi_api_key

# --------------------------
# Initialize SerpAPI wrapper
# --------------------------
search = SerpAPIWrapper()  # automatically uses SERPAPI_API_KEY

# --------------------------
# Safe search function
# --------------------------
def safe_news_search(query: str) -> str:
    """
    Searches the web using SerpAPI and returns a concise summary of the top results.

    Args:
        query (str): The search query. Must not be empty.

    Returns:
        str: Concise summary of the search results or an error message if input is invalid.
    """
    if not query or query.strip() == "":
        return "❌ Cannot perform search: the query is empty."

    # Get top 5 results for brevity
    results = search.results(query)
    organic = results.get("organic_results", [])

    if not organic:
        return "No results found for your query."

    summary_lines = []
    for i, r in enumerate(organic[:5], 1):
        title = r.get("title", "No title")
        link = r.get("link", "No link")
        snippet = r.get("snippet", "")
        summary_lines.append(f"{i}. {title}\n   {snippet}\n   🔗 {link}")

    return "\n\n".join(summary_lines)

# --------------------------
# Wrap as a LangChain tool
# --------------------------
news_tool = Tool(
    name="Latest AI News",
    func=safe_news_search,
    description=(
        "Use this tool to search the web for factual information, news, or updates on a specific topic. "
        "Only call it when the user asks a factual question or requests the latest news. "
        "Do NOT use it for greetings, casual conversation, or personal opinions. "
        "The input must be a valid search query; empty queries will return an error."
    )
)

print("✅ Latest AI News tool is loaded")


✅ Latest AI News tool is loaded


In [19]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=66b22d2ecb68e03b59d6396d7d3c84f4cb3c20d737012bd583a57205c72c7b81
  Stored in directory: /root/.cache/pip/wheels/6e/62/11/dc73d78e40a218ad52e7451f30166e94491be013a7850b5d75
Successfully built fpdf


In [20]:
from fpdf import FPDF

# =================================================
# STEP - 10 :  SAVE CHAT AS PDF  T O O L
# =================================================

def save_chat_as_pdf(chat_text: str, filename="chat.pdf"):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.multi_cell(0, 10, chat_text)
    pdf.output(filename)
    return f"Chat saved to {filename}"

from langchain.agents import Tool

save_pdf_tool = Tool(
    name="SaveChatPDF",
    func=save_chat_as_pdf,
    description="Saves the current chat as a PDF file."
)



In [21]:
# =================================================
# STEP - 10 :  S U M M A R I S E   T O O L
# =================================================

import requests
from bs4 import BeautifulSoup

def summarize_url(url: str):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")
    text = soup.get_text()
    # Use your LLM to summarize
    summary = llm.predict(f"Summarize this text:\n{text}")
    return summary

url_summary_tool = Tool(
    name="URLSummary",
    func=summarize_url,
    description="Reads a URL and returns a summarized version of its content."
)


In [22]:
# ============================================
# STEP - 11 : G R E E T I N G   T O O L
# ============================================

import random # Import the random module

def greet_user(user_input: str = None) -> str:
    greetings_general = [
        "Hello! 👋 I’m your AI Content Coach. How can I assist you today?",
        "Hi there! I’m here to help you create amazing content. What would you like to work on?",
        "Hey! Ready to improve your content? Let’s get started.",
        "Hello! I can guide you through content creation, AI tips, or signature design. What’s first?"
    ]

    if user_input:
        user_input_lower = user_input.lower()
        if any(word in user_input_lower for word in ["morning", "afternoon", "evening"]):
            time_word = next((w for w in ["morning", "afternoon", "evening"] if w in user_input_lower), "day")
            return f"Good {time_word.capitalize()}! I’m your AI Content Coach. How can I help you today?"
        if any(word in user_input_lower for word in ["hi", "hello", "hey"]):
            return random.choice(greetings_general)

    return random.choice(greetings_general)

greeting_tool = Tool(
    name="Greeting",
    func=greet_user,
    description="Responds naturally to greetings like 'hi', 'hello', or 'good morning'."
)


print("✅ Greeting tool is loaded")

✅ Greeting tool is loaded


**================  AGENT  ===================**

In [25]:
# ============================================
# STEP - 12 : Conversational A G E N T
# ============================================

from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from gtts import gTTS
from IPython.display import Audio, display
import openai
# import sounddevice as sd
import numpy as np
import tempfile
import wave


# ----------------------------------------
# Integrating list of tools to the Agent
# ----------------------------------------

tools = [search_tool, greeting_tool, retriever_tool,  whisper_tool, news_tool, tts_tool, save_pdf_tool, url_summary_tool]


# ------------------------------------
# 3️⃣ Initialize Agent with Memory
# ------------------------------------

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)

print("🤖 AI Content Coach is ready! Type 'exit' to quit.")


# --------------------------
# 4️⃣ Run Conversation Loop
# --------------------------

voice_enabled = False  # 🔇 Default OFF

while True:
    user_input = input("\nYou: ")
    if user_input.lower() in ["exit", "quit"]:
        print("👋 Goodbye!")
        break

    # Toggle voice manually
    if user_input.lower() == "voice on":
        voice_enabled = True
        print("🔊 Voice enabled.")
        continue
    elif user_input.lower() == "voice off":
        voice_enabled = False
        print("🔇 Voice disabled.")
        continue

    # Agent responds
    response = agent.run(user_input)
    print("\nAI:", response)

    # Play audio only if enabled
    if voice_enabled:
        text_to_speech(response)

/tmp/ipython-input-87304444.py:27: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


🤖 AI Content Coach is ready! Type 'exit' to quit.

You: Hello


> Entering new AgentExecutor chain...
```json
{
    "action": "Greeting",
    "action_input": "Hello!"
}
```
Observation: Hey! Ready to improve your content? Let’s get started.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Hey! Ready to improve your content? Let’s get started."
}
```

> Finished chain.

AI: Hey! Ready to improve your content? Let’s get started.

You: I want to become an AI content creator


> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "To become an AI content creator, start by learning about AI technologies, tools, and platforms. Familiarize yourself with machine learning concepts, natural language processing, and content generation techniques. You can take online courses, read books, and follow industry news to stay updated. Additionally, practice creating content using AI tools, and consider building a portfolio to showcase your 

In [ ]:
# ============================================
# STEP 13 : S A V E  T H E  A G E N T
# ============================================

import os
import json
import dill
from langchain.vectorstores import Chroma
from langchain.memory import ConversationBufferMemory

# --- Paths ---
agent_dir = "/content/drive/MyDrive/Ironhack_final_project/agent_save"
os.makedirs(agent_dir, exist_ok=True)

# --- 1. Save Tools  ---
def save_tools_config(tools, agent_dir):
    tools_config = []
    for t in tools:
        tools_config.append({
            "name": t.name,
            "description": t.description,
            "func_name": t.func.__name__  # just save function name
        })
    with open(os.path.join(agent_dir, "tools_config.json"), "w") as f:
        json.dump(tools_config, f, indent=2)
    print(f"✅ Tools config saved to {os.path.join(agent_dir, 'tools_config.json')}")

# 🔹 Call it here (assuming you have a `tools` list in your notebook)
if "tools" in globals():
    save_tools_config(tools, agent_dir)
else:
    print("⚠️ No tools found. Tools config not saved.")

# --- 2. Save LLM config ---
llm_config = {
    "model_name": "gpt-4o-mini",
    "temperature": 0.3
}
with open(os.path.join(agent_dir, "llm_config.json"), "w") as f:
    json.dump(llm_config, f, indent=2)
print(f"✅ LLM configuration saved to {os.path.join(agent_dir, 'llm_config.json')}")

# --- 3. Save Memory ---
if "memory" in globals() and isinstance(memory, ConversationBufferMemory):
    try:
        chat_history_state = memory.load_memory_variables({})
        with open(os.path.join(agent_dir, "chat_history.pkl"), "wb") as f:
            dill.dump(chat_history_state, f)
        print(f"✅ Chat history saved to {os.path.join(agent_dir, 'chat_history.pkl')}")
    except Exception as e:
        print(f"❌ Error saving chat history: {e}")
else:
    print("⚠️ Memory object not found or invalid. Chat history not saved.")

# --- 4. Persist Chroma Vectorstore ---
if "vectorstore" in globals() and isinstance(vectorstore, Chroma):
    try:
        vectorstore.persist()
        print("✅ Chroma vector store persisted.")
    except Exception as e:
        print(f"❌ Error persisting Chroma vector store: {e}")
else:
    print("⚠️ Vector store not found or invalid. Not persisted.")

print("🎉 Agent saved successfully!")


✅ Tools config saved to /content/drive/MyDrive/Ironhack_final_project/agent_save/tools_config.json
✅ LLM configuration saved to /content/drive/MyDrive/Ironhack_final_project/agent_save/llm_config.json
✅ Chat history saved to /content/drive/MyDrive/Ironhack_final_project/agent_save/chat_history.pkl
✅ Chroma vector store persisted.
🎉 Agent saved successfully!


/tmp/ipython-input-2839597089.py:58: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


**=================== > > > LANGSMITH EVALUATION < < < ======================**

In [26]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()


In [27]:
%pip install langsmith


In [ ]:
from langsmith import Client
import os

# LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")      #uncomment and run either one according to notebook if not already
LANGCHAIN_API_KEY = userdata.get("LANGCHAIN_API_KEY")

client = Client(api_key=LANGCHAIN_API_KEY)


In [28]:
# Now set the environment for LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"      # enable tracing
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "AI_Content_Coach"  # custom project name

print("✅ LangSmith tracing enabled and API key loaded securely.")

✅ LangSmith tracing enabled and API key loaded securely.


In [39]:
"""
langsmith_eval_manual.py
------------------------
Evaluate your agent with LangSmith using manually defined Q–A pairs.
"""

import logging
from typing import Dict, Any

from langsmith import Client
from langchain.chat_models import ChatOpenAI

from openevals.llm import create_llm_as_judge
from openevals.prompts import (
    CORRECTNESS_PROMPT,
    HALLUCINATION_PROMPT,
    CONCISENESS_PROMPT,
)

# ---------------- CONFIG ----------------
DATASET_NAME = "Agent Evaluation Dataset"
LLM_MODEL = "gpt-4o"          # Model used to generate answers if no agent provided
JUDGE_MODEL = "openai:o3-mini"  # Model used for evaluation
MAX_OUTPUT_CHARS = 600
MAX_COMPLETION_TOKENS = 256
# ----------------------------------------

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# --------- Define your examples manually ---------
examples = [
    {
        "inputs": {"question": "What is the capital of France?"},
        "outputs": {"answer": "Paris"}
    },
    {
        "inputs": {"question": "Who wrote Hamlet?"},
        "outputs": {"answer": "William Shakespeare"}
    },
    {
        "inputs": {"question": "What is 2 + 2?"},
        "outputs": {"answer": "4"}
    },
    # 👉 Add more Q–A pairs here
]

# -------- Target function --------
def truncate_text(s: str, max_chars: int = MAX_OUTPUT_CHARS) -> str:
    return s if len(s) <= max_chars else s[:max_chars] + "...[TRUNCATED]"

def make_target(agent=None):
    """Return a function that LangSmith will evaluate against the dataset."""
    llm = ChatOpenAI(model=LLM_MODEL, temperature=0)

    def target(inputs: Dict[str, Any]) -> Dict[str, Any]:
        question = inputs.get("question", "")
        answer = ""

        if agent:
            try:
                answer = agent.run(question)
            except Exception:
                try:
                    answer = agent.predict(question)
                except Exception:
                    logger.warning("Agent call failed, falling back to LLM.")
                    answer = ""

        if not answer:
            prompt = f"Answer concisely:\n\nQ: {question}\nA:"
            try:
                answer = llm.predict(prompt, max_tokens=MAX_COMPLETION_TOKENS)
            except TypeError:
                answer = llm.predict(prompt)

        return {"answer": truncate_text(answer), "contexts": []}

    return target

# -------- Evaluators --------
def make_evaluators():
    correctness = create_llm_as_judge(
        prompt=CORRECTNESS_PROMPT, feedback_key="correctness", model=JUDGE_MODEL
    )
    hallucination = create_llm_as_judge(
        prompt=HALLUCINATION_PROMPT, feedback_key="hallucination", model=JUDGE_MODEL
    )
    conciseness = create_llm_as_judge(
        prompt=CONCISENESS_PROMPT, feedback_key="conciseness", model=JUDGE_MODEL
    )

    def correctness_eval(run, example):
        return correctness(
            inputs=example.inputs["question"],
            outputs=run.outputs["answer"],
            reference_outputs=example.outputs["answer"]
        )

    def hallucination_eval(run, example):
        return hallucination(
            inputs=example.inputs["question"],
            outputs=run.outputs["answer"],
            context="\n".join(run.outputs.get("contexts", []))
        )

    def conciseness_eval(run, example):
        return conciseness(
            inputs=example.inputs["question"],
            outputs=run.outputs["answer"]
        )

    return [correctness_eval, hallucination_eval, conciseness_eval]

# -------- Main --------
if __name__ == "__main__":
    # 1. Initialize client
    client = Client()

    # 2. Create or update dataset
    if not client.has_dataset(dataset_name=DATASET_NAME):
        client.create_dataset(dataset_name=DATASET_NAME, description="Manual QA pairs")
    client.create_examples(dataset_name=DATASET_NAME, examples=examples)

    # 3. Build target function (replace agent=None with your agent if available)
    target = make_target(agent=None)

    # 4. Get evaluators
    evaluators = make_evaluators()

    # 5. Run evaluation
    results = client.evaluate(
        target,
        data=DATASET_NAME,
        evaluators=evaluators,
        description="Evaluation with correctness, hallucination, conciseness",
        blocking=True
    )

    # 6. Show results
    print("Evaluation complete. View results in LangSmith UI:")
    try:
        print(results.url)
    except:
        print(results)


ModuleNotFoundError: No module named 'openevals'

THE ONE THAT WORKS

In [32]:
# --- Module Docstring ---
"""
This script creates a question-answer dataset for AI content creation in LangSmith,
fetches the API key from Colab secrets, sets environment variables, defines specific
Q&A pairs, uploads them to LangSmith, and evaluates agent responses using multiple
custom evaluators (accuracy, hallucination, document relevance).
"""

# --- Standard library imports ---
import logging
import os
from typing import List, Dict, Any
from datetime import datetime
import asyncio

# --- Third-party imports ---
from langsmith import Client
from langsmith.evaluation import aevaluate
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from dotenv import load_dotenv

# --- Colab secret handling ---
from google.colab import userdata

# --- Load environment variables from .env file (optional for local dev) ---
# load_dotenv()

# --- Configure logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Load LangSmith API key from Colab secrets ---
LANGCHAIN_API_KEY = userdata.get("LANGCHAIN_API_KEY")
if not LANGCHAIN_API_KEY:
    raise ValueError("❌ No LangSmith API key found in Colab secrets!")

# --- Set environment variables for LangSmith ---
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
logging.info("✅ LangSmith API key loaded and environment configured.")

# --- Initialize LangSmith Client ---
# Explicitly initialize the client here to ensure it's available in this cell's scope
try:
    client = Client(api_key=LANGCHAIN_API_KEY)
except Exception as e:
    logging.error(f"Failed to initialize LangSmith client: {e}")
    raise


# --------------------------------------------------
# --- Dataset Creation with Conflict Handling ---
# --------------------------------------------------


DATASET_NAME = "rag_eval_dataset"

try:
    # Use the langsmith client initialized in this cell
    dataset = client.read_dataset(dataset_name=DATASET_NAME)
    logging.info(f"⚠️ Dataset '{DATASET_NAME}' already exists. Using existing dataset.")
except Exception as e:
    logging.info(f"Dataset '{DATASET_NAME}' not found. Creating a new one...")
    # Use the langsmith client initialized in this cell
    dataset = client.create_dataset(
        dataset_name=DATASET_NAME,
        description="RAG evaluation dataset for AI content creation"
    )
    logging.info(f"✅ Dataset '{DATASET_NAME}' created successfully.")

In [33]:
dataset

Dataset(name='rag_eval_dataset', description='A dataset for evaluating a chatbot on creating an AI content creation and the tools required with questions and ideal answers.', data_type=<DataType.kv: 'kv'>, id=UUID('0c368c5b-f89b-47f6-b951-0836d5a806f3'), created_at=datetime.datetime(2025, 9, 12, 17, 54, 47, 395828, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2025, 9, 12, 17, 54, 47, 395828, tzinfo=datetime.timezone.utc), example_count=55, session_count=14, last_session_start_time=datetime.datetime(2025, 9, 18, 18, 57, 38, 512980), inputs_schema=None, outputs_schema=None, transformations=None, metadata=None)

In [34]:
# --- Define Q&A pairs ---
qa_pairs: List[Dict[str, Dict[str, str]]] = [
    {
        "inputs": {
            "question": (
                "Suggest me AI tools for content creation in 2025"
            )
        },
        "outputs": {
            "answer": (
                "Here’s a list of essential AI tools in 2025 for a beginner as AI content creator:\n"
                "Text Generation:** GPT-4/5, Claude 3 - Write articles, scripts, and social media posts.\n"
                "Image & Video Creation:** MidJourney v6, DALL·E 3, Runway - Generate visuals, videos, and thumbnails.\n"
                "Audio & Voice:** ElevenLabs, Descript - Create voiceovers, podcasts, and edit audio.\n"
                "Video Editing:** Runway, CapCut AI - Auto-edit videos, add effects, captions, and summaries."
                "Social Media & Marketing:** Jasper.ai, Buffer AI - Automate posts, optimize engagement, track trends.\n"
                "Research & Idea Generation:** Elicit, Perplexity AI - Quick topic research and summaries.\n"
                "Analytics & Optimization:** VidIQ, Google Analytics AI - Track performance, improve reach and SEO.\n"
                "Workflow & Collaboration:** Notion AI, ClickUp AI - Organize content pipelines and automate tasks.\n"
            )
        }
    },
    {
        "inputs": {
            "question": (
                "Which AI tools can help improve SEO for blogs in 2025?"
            )
        },
        "outputs": {
            "answer": (
                "These tools help optimize content for better search engine ranking and visibility:\n"
                "-Surfer AI: Analyzes keywords and SEO structure to optimize blog content.\n"
                "-Frase: Suggests topic ideas, improves readability, and identifies content gaps.\n"
                "-Writesonic: Generates SEO-friendly content and meta descriptions.\n"
                "-Jasper: Supports keyword integration and content optimization for blogs.\n"

            )
        }
    },
    {
        "inputs": {
            "question": (
                "Which AI tools are good at summarizing long documents or PDFs?"
            )
        },
        "outputs": {
            "answer": (
                "These AI tools extract key points and produce concise summaries:\n"
                "-ChatGPT: Summarizes PDFs, Word documents, and long text efficiently.\n"
                "-Claude: Provides safe and accurate summaries in readable form.\n"
                "-Jasper: Highlights essential information and generates concise overviews.\n"

            )
        }
    },
    {
        "inputs": {
            "question": (
                "What AI tools are useful for social media post creation?"
            )
        },
        "outputs": {
            "answer": (
                "These tools help design graphics, captions, and formatted posts quickly:\n"
                "-Canva Magic Studio: Generates social media graphics and layouts automatically.\n"
                "-Microsoft Designer: Offers AI-assisted design for posts and banners.\n"
                "-Copy.ai: Writes captions, ad copy, and short-form content tailored for social platforms.\n"
                            )
        }
    },
    {
        "inputs": {
            "question": (
                "What AI tools can assist in image generation in 2025?"
            )
        },
        "outputs": {
            "answer": (
                "Answer: These tools create custom visuals, illustrations, and designs from text prompts:\n"
                "-MidJourney: Generates artistic and highly creative images.\n"
                "-DALL·E: Converts text descriptions into realistic or stylized images.\n"
                "-Stable Diffusion: Produces high-quality images for various styles and themes.\n"
                "-Canva Magic Studio: Combines AI image generation with easy-to-use templates for marketing and social media.\n"
            )
        }
    }
]

# --- Add Q&A pairs to dataset ---
for example in qa_pairs:
    client.create_example(inputs=example["inputs"], outputs=example["outputs"], dataset_id=dataset.id)
logging.info(f"✅ Added {len(qa_pairs)} examples to dataset '{DATASET_NAME}'.")

In [ ]:
# # --- Define Q&A pairs ---
# qa_pairs: List[Dict[str, Dict[str, str]]] = [
#     {
#         "inputs": {
#             "question": (
#                 "I’m starting as an AI content creator in 2025. Can you list the most essential AI tools I should use, and explain what each one is best for?"
#             )
#         },
#         "outputs": {
#             "answer": (
#                 "Here’s a list of essential AI tools in 2025 for a beginner as AI content creator:\n"
#                 "Text Generation:** GPT-4/5, Claude 3 - Write articles, scripts, and social media posts."
#                 "Image & Video Creation:** MidJourney v6, DALL·E 3, Runway - Generate visuals, videos, and thumbnails."
#                 "Audio & Voice:** ElevenLabs, Descript - Create voiceovers, podcasts, and edit audio."
#                 "Video Editing:** Runway, CapCut AI - Auto-edit videos, add effects, captions, and summaries."
#                 "Social Media & Marketing:** Jasper.ai, Buffer AI - Automate posts, optimize engagement, track trends."
#                 "Research & Idea Generation:** Elicit, Perplexity AI - Quick topic research and summaries."
#                 "Analytics & Optimization:** VidIQ, Google Analytics AI - Track performance, improve reach and SEO."
#                 "Workflow & Collaboration:** Notion AI, ClickUp AI - Organize content pipelines and automate tasks."
#             )
#         }
#     },
#     {
#         "inputs": {
#             "question": (
#                 "I write long blog articles. How can AI help me repurpose them into short-form content for social media platforms like Instagram, TikTok, and LinkedIn?"
#             )
#         },
#         "outputs": {
#             "answer": (
#                 "AI can help transform long blog articles into engaging short-form content for social media by automatically summarizing key points."
#                 "It can generate platform-specific captions, scripts, or hooks for Instagram Reels, TikTok videos, and LinkedIn posts."
#                 "AI tools can suggest relevant hashtags, keywords, and trending topics to boost visibility and engagement."
#                 "They can also create visual elements like graphics, carousels, or infographics from your blog content."
#                 "This allows you to efficiently repurpose content, save time, and maintain a consistent posting schedule across multiple platforms."

#             )
#         }
#     },
#     {
#         "inputs": {
#             "question": (
#                 "I want to make video content using AI. Which tools should I use for scriptwriting, video generation, and editing, and how do they fit together in a workflow?"
#             )
#         },
#         "outputs": {
#             "answer": (
#                 "AI can help you create video content by streamlining scriptwriting, video generation, and editing in a seamless workflow.\n"
#                 "-For scriptwriting, tools like GPT-4/5 or Jasper.ai can generate engaging scripts, dialogue, or storyboards from your ideas."
#                 "-For video generation, platforms like Runway, Pictory, or Synthesia can turn scripts into AI-generated videos with animations, avatars, or stock footage."
#                 "-For editing, tools like Descript, CapCut AI, or Adobe Firefly can automatically trim, add captions, transitions, and enhance visuals." "A typical workflow"
#                 "starts with AI-generated scripts, moves to video creation, and finishes with AI-assisted editing before publishing on platforms like YouTube, TikTok, or Instagram."

#             )
#         }
#     },
#     {
#         "inputs": {
#             "question": (
#                 "How can AI improve the SEO of my blog posts and ensure they rank higher in Google searches? Can you name specific tools and how they help?"
#             )
#         },
#         "outputs": {
#             "answer": (
#                 "AI-powered SEO tools to boost rankings:\n"
#                 "- SurferSEO: Recommends keywords, structure, and length based on competitors.\n"
#                 "- Clearscope: Uses NLP to include semantically related terms.\n"
#                 "- SEMRush AI Assistant: Tracks keyword rankings and suggests updates.\n"
#                 "- MarketMuse: Identifies content gaps for untapped opportunities.\n"
#                 "- ChatGPT (SEO plugins): Drafts keyword-rich meta descriptions and FAQ sections."
#             )
#         }
#     },
#     {
#         "inputs": {
#             "question": (
#                 "I often struggle to come up with new content ideas. How can AI assist me in brainstorming creative content topics, campaign ideas, or trending formats for 2025?"
#             )
#         },
#         "outputs": {
#             "answer": (
#                 "AI can generate fresh, trend-driven ideas:\n"
#                 "- ChatGPT: Brainstorms article titles, podcast themes, and outlines.\n"
#                 "- Jasper: Creates ad campaign slogans and A/B test variants.\n"
#                 "- Copy.ai: Produces multiple versions of headlines and hooks.\n"
#                 "- BuzzSumo + AI: Finds trending topics, viral posts, and competitor insights.\n"
#                 "- Notion AI: Organizes brainstormed ideas into campaigns, schedules, and task lists."
#             )
#         }
#     }
# ]

# # --- Add Q&A pairs to dataset ---
# for example in qa_pairs:
#     client.create_example(inputs=example["inputs"], outputs=example["outputs"], dataset_id=dataset.id)
# logging.info(f"✅ Added {len(qa_pairs)} examples to dataset '{DATASET_NAME}'.")

In [ ]:
"""
langsmith_agent_eval.py

Drop this into the same environment where you have your agent/retriever
(or run it as a standalone evaluation). Update CONFIG at top for your
paths/model choices.

Dependencies:
    pip install -U langsmith langchain openevals openai pandas
Environment variables:
    LANGSMITH_TRACING=true
    LANGSMITH_API_KEY=...
    OPENAI_API_KEY=...
"""

import os
import math
import textwrap
import logging
from functools import partial
from typing import List, Dict, Any, Optional

import pandas as pd

# LangSmith client + wrappers
from langsmith import Client
# openevals -> LLM-as-judge utilities
from openevals.llm import create_llm_as_judge
from openevals.prompts import CORRECTNESS_PROMPT, HALLUCINATION_PROMPT, CONCISENESS_PROMPT

# Optional LangChain chat model (fallback path)
from langchain.chat_models import ChatOpenAI
from langchain.schema import Document

# ------------- CONFIG ----------------
DATA_PATH = "examples.csv"   # or examples.jsonl; if None you can set `examples_list` below
DATASET_NAME = "My Agent Eval Dataset"
MAX_OUTPUT_CHARS = 800       # final truncation limit for every answer
MAX_COMPLETION_TOKENS = 256  # hint to the LLM provider, still truncate post-hoc
JUDGE_MODEL = "openai:o3-mini"  # model used by openevals judges (low-cost judge)
LLM_MODEL = "gpt-4o"              # fallback LLM if you don't pass an `agent`
TEMPERATURE = 0.0
MAX_CONCURRENCY = 4
# ------------------------------------

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def load_examples_from_csv(path: str) -> List[Dict[str, Any]]:
    """Load CSV with columns 'question' and 'answer' into LangSmith examples format."""
    df = pd.read_csv(path)
    required = {"question", "answer"}
    if not required.issubset(set(df.columns)):
        raise ValueError(f"CSV must contain columns {required}. Found: {list(df.columns)}")
    examples = []
    for _, row in df.iterrows():
        examples.append({
            "inputs": {"question": str(row["question"])},
            "outputs": {"answer": str(row["answer"])},
            "metadata": {}
        })
    return examples


def create_or_update_dataset(client: Client, dataset_name: str, examples: List[Dict[str, Any]]):
    """Create dataset if missing and upload examples (idempotent-ish)."""
    # Create dataset if not present
    if not client.has_dataset(dataset_name=dataset_name):
        logger.info("Creating dataset in LangSmith: %s", dataset_name)
        client.create_dataset(dataset_name=dataset_name, description="Auto-created dataset for agent evaluation")
    else:
        logger.info("Dataset already exists: %s", dataset_name)

    # Upload examples in batches using client.create_examples
    logger.info("Uploading %d examples to dataset %s", len(examples), dataset_name)
    # client.create_examples accepts a list; adapt to the client signature if needed
    client.create_examples(dataset_name=dataset_name, examples=examples)
    logger.info("Upload done.")


def truncate_text(s: str, max_chars: int = MAX_OUTPUT_CHARS) -> str:
    if not isinstance(s, str):
        return s
    if len(s) <= max_chars:
        return s
    return s[:max_chars].rstrip() + "\n\n...[TRUNCATED]"


def make_target(agent=None, retriever=None, llm_fallback: Optional[ChatOpenAI] = None,
                max_completion_tokens: int = MAX_COMPLETION_TOKENS,
                max_output_chars: int = MAX_OUTPUT_CHARS):
    """
    Returns a function target(inputs: dict) -> dict suitable for client.evaluate().
    - If `agent` is provided, the agent will be used (supports `.run(question)` or `.predict(question)`).
    - If not, `retriever` + `llm_fallback` produce a RAG-style answer.
    The returned dict MUST include keys that your evaluators need, e.g. 'answer' and 'contexts'.
    """
    if llm_fallback is None:
        llm_fallback = ChatOpenAI(model=LLM_MODEL, temperature=TEMPERATURE)

    def target(inputs: Dict[str, Any]) -> Dict[str, Any]:
        question = inputs.get("question") or inputs.get("prompt") or ""
        if question is None or question == "":
            return {"answer": "", "contexts": []}

        # Fetch contexts (if retriever available)
        contexts = []
        try:
            if retriever:
                docs: List[Document] = retriever.get_relevant_documents(question)
                contexts = [d.page_content for d in docs]
        except Exception as e:
            logger.warning("Retriever call failed, continuing w/out contexts: %s", e)
            contexts = []

        # Use agent if provided; attempt `.run()` then `.predict()`
        answer = ""
        if agent:
            try:
                # many LangChain agents expose .run()
                answer = agent.run(question)
            except Exception:
                try:
                    answer = agent.predict(question)
                except Exception:
                    logger.exception("Agent call failed, falling back to LLM-only.")
                    answer = ""

        # Fallback: simple instruction prompt + contexts -> llm
        if not answer:
            # Build concise RAG prompt that instructs the model to be brief
            prompt_parts = [
                "You are a concise, factual assistant. Base your answer only on the contexts if they exist.",
                f"Question: {question}",
            ]
            if contexts:
                prompt_parts.append("CONTEXTS:\n" + "\n\n---\n".join(contexts))
                prompt_parts.append("Answer in 1-3 short paragraphs and be concise.")
            else:
                prompt_parts.append("No external contexts provided. Answer concisely.")

            prompt = "\n\n".join(prompt_parts)
            # Ask ChatOpenAI to produce the answer; pass token limit hint (providers may respect)
            try:
                answer = llm_fallback.predict(prompt, max_tokens=max_completion_tokens)
            except TypeError:
                # some LangChain versions use different kwargs naming; try without kwargs
                answer = llm_fallback.predict(prompt)

        # Enforce truncation to avoid super-long outputs in LangSmith
        answer = truncate_text(answer, max_chars=max_output_chars)
        return {"answer": answer, "contexts": contexts}

    return target


# ----------------- Evaluators -----------------
# We'll create LLM-as-judge evaluators using OpenEvals, then wrap them into functions with the
# signature expected by LangSmith's client.evaluate (run, example) -> dict

def make_openevals_judges(judge_model: str = JUDGE_MODEL):
    """Return a small dict of judge callables from openevals"""
    correctness_judge = create_llm_as_judge(
        prompt=CORRECTNESS_PROMPT,
        feedback_key="correctness",
        model=judge_model,
    )
    hallucination_judge = create_llm_as_judge(
        prompt=HALLUCINATION_PROMPT,
        feedback_key="hallucination",
        model=judge_model,
    )
    conciseness_judge = create_llm_as_judge(
        prompt=CONCISENESS_PROMPT,
        feedback_key="conciseness",
        model=judge_model,
    )
    return {
        "correctness": correctness_judge,
        "hallucination": hallucination_judge,
        "conciseness": conciseness_judge
    }


def wrap_correctness(judge_callable):
    """Wrap openevals correctness judge into (run, example) signature."""
    def evaluator(run, example):
        inputs = example.inputs.get("question", "")
        outputs = run.outputs.get("answer", "")
        reference_outputs = example.outputs.get("answer", "")
        try:
            res = judge_callable(inputs=inputs, outputs=outputs, reference_outputs=reference_outputs)
            # openevals returns dict like {'key':..., 'score': bool/float, 'comment': '...'}
            return {"score": bool(res.get("score")), "comment": res.get("comment")}
        except Exception as e:
            logger.exception("Correctness judge failed: %s", e)
            # Fallback heuristics: exact/near-exact string match
            pred = (outputs or "").strip().lower()
            ref = (reference_outputs or "").strip().lower()
            simple_score = 1.0 if pred == ref and pred != "" else 0.0
            return {"score": simple_score, "comment": f"fallback_simple_match={simple_score}"}
    return evaluator


def wrap_hallucination(judge_callable):
    """Judge whether answer contains hallucinated facts not supported by contexts."""
    def evaluator(run, example):
        inputs = example.inputs.get("question", "")
        outputs = run.outputs.get("answer", "")
        # Contexts can be stored either in example.outputs or run.outputs; prefer run.outputs
        contexts = run.outputs.get("contexts") or example.outputs.get("contexts") or []
        context_text = "\n\n---\n".join(contexts) if isinstance(contexts, (list, tuple)) else str(contexts)
        try:
            res = judge_callable(inputs=inputs, outputs=outputs, context=context_text)
            return {"score": bool(res.get("score")), "comment": res.get("comment")}
        except Exception as e:
            logger.exception("Hallucination judge failed: %s", e)
            # fallback: simple heuristic - check if many unique tokens in outputs are missing from context
            out_tokens = set(str(outputs).lower().split())
            ctx_tokens = set(str(context_text).lower().split())
            missing = out_tokens - ctx_tokens
            # proportion of tokens missing (naive)
            prop_missing = len(missing) / max(1, len(out_tokens))
            score = 0.0 if prop_missing > 0.5 else 1.0
            return {"score": bool(score), "comment": f"fallback_prop_missing={prop_missing:.2f}"}
    return evaluator


def wrap_conciseness(judge_callable):
    def evaluator(run, example):
        inputs = example.inputs.get("question", "")
        outputs = run.outputs.get("answer", "")
        try:
            res = judge_callable(inputs=inputs, outputs=outputs)
            return {"score": bool(res.get("score")), "comment": res.get("comment")}
        except Exception as e:
            logger.exception("Conciseness judge failed: %s", e)
            # fallback: prefer shorter outputs; pass if output <= MAX_OUTPUT_CHARS
            is_concise = len((outputs or "")) <= MAX_OUTPUT_CHARS
            return {"score": bool(is_concise), "comment": "fallback_length_check"}
    return evaluator


# ----------------- Runner / Main -----------------

def run_evaluation(
    client: Client,
    target_callable,
    dataset_name: str,
    judges: Dict[str, Any],
    evaluators_to_use: Optional[List[str]] = None,
    blocking: bool = True,
    max_concurrency: int = MAX_CONCURRENCY,
):
    """
    Run client.evaluate with the wrapped evaluators.
    `judges` should be the dict returned by make_openevals_judges()
    """
    if evaluators_to_use is None:
        evaluators_to_use = ["correctness", "hallucination", "conciseness"]

    # Wrap judges into row-evaluators compatible with client.evaluate
    row_evaluators = []
    mapping = {
        "correctness": wrap_correctness,
        "hallucination": wrap_hallucination,
        "conciseness": wrap_conciseness
    }

    for name in evaluators_to_use:
        if name not in judges:
            raise ValueError(f"Unknown evaluator requested: {name}")
        wrapped = mapping[name](judges[name])
        row_evaluators.append(wrapped)

    logger.info("Launching evaluation on dataset: %s", dataset_name)
    experiment_results = client.evaluate(
        target_callable,
        data=dataset_name,
        evaluators=row_evaluators,
        description="Automated evaluation: correctness / hallucination / conciseness",
        max_concurrency=max_concurrency,
        blocking=blocking,
    )
    return experiment_results


# ------------- Example script usage -------------
if __name__ == "__main__":
    # 1) Initialize LangSmith client
    client = Client()  # reads LANGSMITH_API_KEY from env

    # 2) Load examples (either from CSV / JSONL or supply examples_list manually)
    if os.path.exists(DATA_PATH):
        examples_list = load_examples_from_csv(DATA_PATH)
    else:
        # If you already have examples in memory, set them here
        # Example format:
        examples_list = [
            {"inputs": {"question": "What is an LLM?"}, "outputs": {"answer": "A large language model is..."}},
            # ...
        ]
        logger.warning("Using embedded examples_list - no DATA_PATH found at %s", DATA_PATH)

    # 3) Create/upload dataset
    create_or_update_dataset(client, DATASET_NAME, examples_list)

    # 4) Prepare target function:
    # If you already have an `agent` variable in your notebook, import it and pass here.
    # Example: from my_agent_module import agent, retriever
    # Then call make_target(agent=agent, retriever=retriever)
    # For this script we default to fallback LLM only:
    llm_fallback = ChatOpenAI(model=LLM_MODEL, temperature=TEMPERATURE)
    target = make_target(agent=None, retriever=None, llm_fallback=llm_fallback)

    # 5) Create judges
    judges = make_openevals_judges(judge_model=JUDGE_MODEL)

    # 6) Run evaluation (blocking)
    results = run_evaluation(
        client=client,
        target_callable=target,
        dataset_name=DATASET_NAME,
        judges=judges,
        evaluators_to_use=["correctness", "hallucination", "conciseness"],
        blocking=True,
        max_concurrency=MAX_CONCURRENCY
    )

    # 7) Print experiment details and where to view the UI
    try:
        print("Experiment results URL:", results.url)
    except Exception:
        print("Experiment name:", results.experiment_name)

    # Optionally fetch row-level test results (pandas)
    try:
        df = client.get_test_results(project_name=None)  # or use project_name if you set one
        print("Summary of test results (head):")
        print(df.head().to_string())
    except Exception as e:
        logger.info("Could not fetch test results via client.get_test_results(): %s", e)

    print("Done.")


In [36]:
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("❌ No OpenAI API key found in Colab secrets!")

In [37]:
from langsmith import Client
import os

# LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")      #uncomment and run either one according to notebook if not already
LANGCHAIN_API_KEY = userdata.get("LANGCHAIN_API_KEY")

client = Client(api_key=LANGCHAIN_API_KEY)


In [38]:

import os

if "OPENAI_API_KEY" in globals() and OPENAI_API_KEY:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
    logging.info("✅ OPENAI_API_KEY environment variable set in cell 3GJ1BNnKv822.")
else:
    logging.error("❌ OPENAI_API_KEY not found in globals or is empty.")


# --- Async helper to retrieve documents ---
async def get_relevant_documents(query: str) -> List[Document]:
    """
    Retrieves relevant documents using the retriever from chatbot_backend.py.
    """
    if retriever is None:
        raise ValueError("Retriever is None. Ensure chatbot_backend.py initializes it correctly.")
    return await retriever.ainvoke(query)

# --- Load evaluation prompts from LangChain Hub ---
grade_prompt_hallucinations = hub.pull("langchain-ai/rag-answer-hallucination")
grade_prompt_doc_relevance = hub.pull("langchain-ai/rag-document-relevance")
grade_prompt_answer = hub.pull("langchain-ai/rag-answer-vs-reference")

# --- Custom Evaluator Functions ---



async def answer_evaluator(run, example) -> dict:
    """
    Evaluates the accuracy of the generated answer against reference answer.
    """
    input_question = example.inputs.get("question", "")
    reference_answer = example.outputs.get("answer", "")
    prediction = run.outputs.get("answer", "")

    llm = ChatOpenAI(model="gpt-4o", temperature=0)
    answer_grader = grade_prompt_answer | llm

    try:
        # Correct input keys for rag-answer-vs-reference
        score_response = await answer_grader.ainvoke({
            "question": input_question,
            "reference_answer": reference_answer,
            "predicted_answer": prediction
        })
        # Parse output as string
        score_text = str(score_response).strip().lower()
        score = 1.0 if score_text in ["yes", "y", "true", "t"] else 0.0

    except Exception as e:
        logging.error(f"Error grading answer accuracy: {e}", exc_info=True)
        score = 0.0

    print(f'question="{input_question}", accuracy_score={score}')
    return {"key": "answer_accuracy", "score": score}


async def answer_hallucination_evaluator(run, example) -> dict:
    """
    Evaluates if the generated answer contains hallucinated information.
    """
    input_question = example.inputs.get("question", "")
    prediction = run.outputs.get("answer", "")
    contexts = run.outputs.get("contexts", [])

    llm = ChatOpenAI(model="gpt-4o", temperature=0)
    hallucination_grader = grade_prompt_hallucinations | llm

    try:
        # Correct input keys for rag-answer-hallucination
        score_response = await hallucination_grader.ainvoke({
            "answer": prediction,
            "sources": contexts
        })
        score_text = str(score_response).strip().lower()
        score = 0.0 if score_text in ["yes", "y", "true", "t"] else 1.0
        # Here, 1 = no hallucination, 0 = hallucination detected

    except Exception as e:
        logging.error(f"Error grading hallucination: {e}", exc_info=True)
        score = 0.0

    print(f'question="{input_question}", hallucination_score={score}')
    return {"key": "answer_hallucination", "score": score}


async def docs_relevance_evaluator(run, example) -> dict:
    """
    Evaluates if the retrieved documents are relevant to the input question.
    """
    input_question = example.inputs.get("question", "")
    contexts = run.outputs.get("contexts", [])

    llm = ChatOpenAI(model="gpt-4o mini", temperature=0)
    relevance_grader = grade_prompt_doc_relevance | llm

    try:
        # Correct input keys for rag-document-relevance
        score_response = await relevance_grader.ainvoke({
            "question": input_question,
            "documents": contexts
        })
        score_text = str(score_response).strip().lower()
        score = 1.0 if score_text in ["yes", "y", "true", "t"] else 0.0

    except Exception as e:
        logging.error(f"Error grading document relevance: {e}", exc_info=True)
        score = 0.0

    print(f'question="{input_question}", doc_relevance_score={score}')
    return {"key": "document_relevance", "score": score}



# --- Async Prediction Function ---
async def predict_agent_run_for_eval(example: Dict[str, Any]) -> Dict[str, Any]:
    """
    Uses agent to answer a question and provide retrieved contexts.
    """
    question = example.get("question")
    if not question:
        logging.error("Question key missing from example inputs.")
        return {"answer": "Error: Question missing.", "contexts": []}

    logging.info(f"Evaluating question with agent: {question}")

    # Retrieve documents asynchronously
    retrieved_docs_strings = []
    try:
        retrieved_docs = await get_relevant_documents(question)
        retrieved_docs_strings = [doc.page_content for doc in retrieved_docs]
    except Exception as e:
        logging.warning(f"Failed to retrieve documents: {e}")

    # Generate answer asynchronously
    try:
        response = await agent.ainvoke({"input": question})
        final_answer = response.get("output", "No answer generated.")
        return {"answer": final_answer, "contexts": retrieved_docs_strings}
    except Exception as e:
        logging.error(f"Error during agent execution: {e}", exc_info=True)
        return {"answer": f"Error during agent execution: {e}", "contexts": retrieved_docs_strings}


# --- Run Evaluation ---
dataset_name = DATASET_NAME  # Ensure this matches the dataset created above

async def main():
    experiment_results = await aevaluate(
        predict_agent_run_for_eval,
        data=dataset_name,
        evaluators=[
            answer_evaluator,
            answer_hallucination_evaluator,
            docs_relevance_evaluator
        ],
        experiment_prefix=f"RAG_AI_Content_Eval_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
        metadata={
            "variant": "Evaluation with specific questions from LangSmith dataset",
            "source_dataset_name": dataset_name
        },
    )

    print("\n--- Evaluation complete! ---")
    try:
        print(f"View results on LangSmith: {experiment_results.url}")
    except AttributeError:
        print(f"Experiment name: {experiment_results.experiment_name}")


if __name__ == "__main__":
    await main()

View the evaluation results for experiment: 'RAG_AI_Content_Eval_20250919_075218-7d81983d' at:
https://smith.langchain.com/o/e11e364c-ad8b-4e26-9bf3-bf04a01e332f/datasets/0c368c5b-f89b-47f6-b951-0836d5a806f3/compare?selectedSessions=72a83842-d5eb-4b9a-9639-902d497cc7e4




0it [00:00, ?it/s]



> Entering new AgentExecutor chain...
```json
{
    "action": "Chroma Retriever",
    "action_input": "AI tools for image generation in 2025"
}
```
Observation: [Document(metadata={'chunk_index': 0, 'source': 'S2TAa4P2IuY'}, page_content="AI has revolutionized image creation. You can now type in a text prompt, like design a storefront for a cookie store, and get captivating visuals back. There are lots of different AI image generators out there, both free and also paid, and you've probably used some yourself. In this video, in partnership with Zapier, we'll look at the top seven AI image generators from Dalle 3 to Midjourney and more. Let's dive in. First up, we have Dalle 3, which is developed by OpenAI, the maker of ChatGPT. You can access this directly from chatgpt.com with a free account, although you only get two images per day. With a plus account, on the other hand, you can create more images, but plans start at $20 per month. As an alternative, you can go to Microsoft's Bing 

ERROR:root:Error grading answer accuracy: "Input to StructuredPrompt is missing variables {'student_answer', 'correct_answer'}.  Expected: ['correct_answer', 'question', 'student_answer'] Received: ['question', 'reference_answer', 'predicted_answer']\nNote: if you intended {student_answer} to be part of the string and not a variable, please escape it with double curly braces like: '{{student_answer}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "
Traceback (most recent call last):
  File "/tmp/ipython-input-3675071000.py", line 41, in answer_evaluator
    score_response = await answer_grader.ainvoke({
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langchain_core/runnables/base.py", line 3290, in ainvoke
    input_ = await coro_with_context(part(), context, create_task=True)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12

```json
{
    "action": "Final Answer",
    "action_input": "In 2025, several AI tools can assist in image generation, including DALL-E 3, which allows users to create images from text prompts and offers API access for automation. Microsoft's Bing Image Creator also utilizes DALL-E 3 and allows for unlimited image creation for free. Midjourney is known for its high-quality and vibrant images, though it operates on a subscription basis. Stable Diffusion is another popular option for generating images."
}
```

> Finished chain.
question="What AI tools can assist in image generation in 2025?", accuracy_score=0.0
question="What AI tools can assist in image generation in 2025?", hallucination_score=0.0
question="What AI tools can assist in image generation in 2025?", doc_relevance_score=0.0


> Entering new AgentExecutor chain...


...

```json
{
    "action": "Final Answer",
    "action_input": "To create video content using AI, consider the following tools: 1. **InVideo**: Great for scriptwriting and video creation with templates and stock footage. 2. **Synthesia**: Ideal for generating AI-driven videos with avatars and voiceovers. 3. **Murf**: Excellent for creating human-like voiceovers for your videos. 4. **Descript**: Useful for video editing, allowing you to edit video by editing text. 5. **Canva**: For creating engaging thumbnails and social media graphics. **Workflow**: Start by brainstorming ideas and writing scripts using InVideo or ChatGPT. Then, create videos using Synthesia or InVideo. Add voiceovers with Murf, and finally, edit the video using Descript. Use Canva for promotional graphics to share on social media."
}
```

> Finished chain.
question="I want to make video content using AI. Which tools should I use for scriptwriting, video generation, and editing, and how do they fit together in a workflow?

ERROR:root:Error grading answer accuracy: "Input to StructuredPrompt is missing variables {'student_answer', 'correct_answer'}.  Expected: ['correct_answer', 'question', 'student_answer'] Received: ['question', 'reference_answer', 'predicted_answer']\nNote: if you intended {student_answer} to be part of the string and not a variable, please escape it with double curly braces like: '{{student_answer}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "
Traceback (most recent call last):
  File "/tmp/ipython-input-3675071000.py", line 41, in answer_evaluator
    score_response = await answer_grader.ainvoke({
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langchain_core/runnables/base.py", line 3290, in ainvoke
    input_ = await coro_with_context(part(), context, create_task=True)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12

```json
{
    "action": "Final Answer",
    "action_input": "AI can help you repurpose long blog articles into short-form content by using tools like Copy.ai to generate concise summaries and engaging captions, Canva for creating visually appealing graphics, and InVideo for transforming written content into short videos. You can extract key points from your articles using AI summarization tools, then adapt these points into formats suitable for each platform, such as infographics for Instagram, short clips for TikTok, and concise posts for LinkedIn."
}
```

> Finished chain.
question="I write long blog articles. How can AI help me repurpose them into short-form content for social media platforms like Instagram, TikTok, and LinkedIn?", accuracy_score=0.0
question="I write long blog articles. How can AI help me repurpose them into short-form content for social media platforms like Instagram, TikTok, and LinkedIn?", hallucination_score=0.0
question="I write long blog articles. How can AI 

ERROR:root:Error grading answer accuracy: "Input to StructuredPrompt is missing variables {'student_answer', 'correct_answer'}.  Expected: ['correct_answer', 'question', 'student_answer'] Received: ['question', 'reference_answer', 'predicted_answer']\nNote: if you intended {student_answer} to be part of the string and not a variable, please escape it with double curly braces like: '{{student_answer}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "
Traceback (most recent call last):
  File "/tmp/ipython-input-3675071000.py", line 41, in answer_evaluator
    score_response = await answer_grader.ainvoke({
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langchain_core/runnables/base.py", line 3290, in ainvoke
    input_ = await coro_with_context(part(), context, create_task=True)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12

```json
{
    "action": "Final Answer",
    "action_input": "In 2025, essential AI tools for content creation include: 1. **ChatGPT** - Great for generating ideas and drafting content. 2. **Jasper** - Excellent for creating marketing copy and blog posts. 3. **Copy.ai** - Useful for brainstorming and generating social media content. 4. **Canva** - Ideal for designing visuals and graphics. 5. **InVideo** - Best for creating video content from scripts. 6. **Murf** - Provides voiceovers for videos. 7. **Descript** - Useful for editing video and audio content. 8. **Surfer SEO** - Helps optimize content for search engines. 9. **Frase** - Assists in creating SEO-friendly content by analyzing competitors. 10. **SEMrush** - Comprehensive tool for keyword research and SEO analysis."
}
```

> Finished chain.
question="I’m starting as an AI content creator in 2025. Can you list the most essential AI tools I should use, and explain what each one is best for?", accuracy_score=0.0
question="I’m start

In [ ]:
import asyncio
from langsmith import Client
import os
from datetime import datetime
from langchain.agents import initialize_agent, AgentType
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.tools import Tool
from typing import Dict, Any
import random

# Assuming 'client', 'dataset', 'tools', 'llm', 'memory', 'DATASET_NAME', and 'OPENAI_API_KEY'
# are defined in previously executed cells.

# --- Define Agent Factory ---
def create_agent():
    """
    Factory function that initializes and returns the agent.
    Assumes 'tools', 'llm', and 'memory' are available in the global scope.
    """
    # Ensure required components are available
    if 'tools' not in globals():
        raise ValueError("Global 'tools' list not found.")
    if 'llm' not in globals():
         raise ValueError("Global 'llm' object not found.")
    # The memory object is created within the factory, so no need to check global scope

    # Re-initialize the agent within the factory
    agent_instance = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
        memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True), # Initialize memory here if needed per run
        verbose=True,
        handle_parsing_errors=True
    )
    return agent_instance

# --- Input Mapper for Dataset ---
def map_inputs(example_inputs: Dict[str, Any]) -> Dict[str, Any]:
    """Maps dataset example inputs ('question') to the agent's expected input format ('input')."""
    return {"input": example_inputs.get("question", "")}


# --- Run Agent on Dataset ---
async def run_evaluation_dataset():
    """
    Runs the agent on the specified dataset and logs results to LangSmith.
    """
    if 'client' not in globals() or client is None:
         raise ValueError("LangSmith client not found or is None. Ensure it's initialized.")
    if 'dataset' not in globals() or dataset is None:
         raise ValueError("Dataset object not found or is None. Ensure it's created in a previous cell.")
    if 'DATASET_NAME' not in globals():
         raise ValueError("DATASET_NAME is not defined.")


    print(f"Running agent on dataset '{dataset.name}'...")

    # Use client.run_on_dataset with the factory function and input_mapper
    # Make the experiment prefix more unique
    experiment_prefix=f"RunOnDataset_AI_Content_Eval_{datetime.now().strftime('%Y%m%d_%H%M%S')}_{random.randint(1000, 9999)}"
    project_name=os.environ.get("LANGCHAIN_PROJECT", "default")

    results = await client.run_on_dataset(
        dataset_name=DATASET_NAME,
        llm_or_chain_factory=create_agent, # Pass the factory function
        input_mapper=map_inputs, # Pass the input mapper function
        experiment_prefix=experiment_prefix,
        project_name=project_name,
        concurrency=1, # Start with concurrency 1 to avoid potential memory issues with ConversationBufferMemory per run
        verbose=True,
    )

    print("\n--- Evaluation run complete! ---")
    try:
        if hasattr(results, 'url'):
             print(f"View results on LangSmith: {results.url}")
        elif isinstance(results, list) and len(results) > 0 and hasattr(results[0], 'url'):
             # Fallback for older versions or different structures
             print(f"View results on LangSmith (first run URL): {results[0].url.rsplit('/', 1)[0]}") # Get experiment URL from run URL
        else:
             print("Results logged to LangSmith. Check your project dashboard.")
    except Exception as e:
        print(f"Could not generate LangSmith URL: {e}")
        print("Results logged to LangSmith. Check your project dashboard.")

# Run the async function
await run_evaluation_dataset()

In [ ]:
# # --- Module Docstring ---
# """
# This script creates a question-answer dataset for AI content creation in LangSmith,
# fetches the API key from Colab secrets, sets environment variables, defines specific
# Q&A pairs, uploads them to LangSmith, and evaluates agent responses using multiple
# custom evaluators (accuracy, hallucination, document relevance).
# """

# # --- Standard library imports ---
# import logging
# import os
# from typing import List, Dict, Any
# from datetime import datetime
# import asyncio

# # --- Third-party imports ---
# from langsmith import Client
# from langsmith.evaluation import aevaluate
# from langchain import hub
# from langchain_openai import ChatOpenAI
# from langchain_core.documents import Document
# from dotenv import load_dotenv
# from langchain_core.runnables import RunnableLambda # Import RunnableLambda

# # --- Colab secret handling ---
# from google.colab import userdata

# # --- Load environment variables from .env file (optional for local dev) ---
# load_dotenv()

# # --- Configure logging ---
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# # --- Load LangSmith API key from Colab secrets ---
# langsmith_api_key = userdata.get("LANGCHAIN_API_KEY")
# if not langsmith_api_key:
#     raise ValueError("❌ No LangSmith API key found in Colab secrets!")

# # --- Set environment variables for LangSmith ---
# os.environ["LANGCHAIN_API_KEY"] = langsmith_api_key
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# logging.info("✅ LangSmith API key loaded and environment configured.")

# # --- Initialize LangSmith Client ---
# client = Client()

# # --- Dataset Creation with Conflict Handling ---
# DATASET_NAME = "rag_eval_dataset"

# try:
#     dataset = client.read_dataset(dataset_name=DATASET_NAME)
#     logging.info(f"⚠️ Dataset '{DATASET_NAME}' already exists. Using existing dataset.")
# except Exception as e:
#     logging.info(f"Dataset '{DATASET_NAME}' not found. Creating a new one...")
#     dataset = client.create_dataset(
#         dataset_name=DATASET_NAME,
#         description="RAG evaluation dataset for AI content creation"
#     )
#     logging.info(f"✅ Dataset '{DATASET_NAME}' created successfully.")


# --- Import chatbot backend components ---
# try:
#     from chatbot_backend import agent, retriever
#     logging.info("Successfully imported agent and retriever from chatbot_backend.py")
# except ImportError as e:
#     logging.error(f"Failed to import components from chatbot_backend.py: {e}")
#     exit("Exiting: Could not load core chatbot components. Check chatbot_backend.py.")

# Assuming agent and retriever are defined in previous cells in this notebook
# Ensure the agent object is named 'agent' and the retriever is named 'retriever'

# --- Async helper to retrieve documents ---
async def get_relevant_documents(query: str, retriever_instance) -> List[Document]:
    """
    Retrieves relevant documents using the provided retriever instance.
    """
    if retriever_instance is None:
         raise ValueError("Retriever instance is None.")
    return await retriever_instance.ainvoke(query)

# --- Load evaluation prompts from LangChain Hub ---
grade_prompt_hallucinations = hub.pull("langchain-ai/rag-answer-hallucination")
grade_prompt_doc_relevance = hub.pull("langchain-ai/rag-document-relevance")
grade_prompt_answer = hub.pull("langchain-ai/rag-answer-vs-reference")

# --- Custom Evaluator Functions ---

def answer_evaluator(run, example) -> dict:
    """
    Evaluates the accuracy of the generated answer against reference answer.
    """
    input_question = example.inputs.get("question", "")
    correct_answer = example.outputs.get("answer", "")
    prediction = run.outputs.get("answer", "")

    llm = ChatOpenAI(model="gpt-4o", temperature=0)
    answer_grader = grade_prompt_answer | llm

    try:
        # LangChain Expression Language (LCEL) invoke
        score_response = answer_grader.invoke({
            "question": input_question,
            "correct_answer": correct_answer,
            "student_answer": prediction
        })
        # Extracting the score from the response structure
        # Assuming the response has a 'Score' key with 'YES' or 'NO'
        raw_score_text = score_response.content.strip().lower() # Access content attribute for ChatMessage
        score = 1.0 if raw_score_text.startswith("yes") else 0.0 # Check if it starts with 'yes'
    except Exception as e:
        logging.error(f"Error grading answer accuracy: {e}", exc_info=True)
        score = 0.0

    print(f'question="{input_question}", score={score}')
    return {"key": "answer_accuracy", "score": score}


def answer_hallucination_evaluator(run, example) -> dict:
    """
    Evaluates if the generated answer contains hallucinated information.
    """
    input_question = example.inputs.get("question", "")
    # Ensure 'contexts' is correctly extracted from the run outputs
    # The agent run might return outputs in a different structure
    # Based on the agent definition, the output might be in 'output' key, not 'contexts'
    # If contexts are needed, the agent run structure needs to provide them
    # For this evaluator, we need the retrieved documents which might not be in agent.run output directly
    # A better approach might be to retrieve contexts separately or modify the agent output
    # For now, let's assume the agent run output contains 'contexts' key from the retriever tool's output
    # Correcting to access contexts from the custom prediction function's output
    contexts = run.outputs.get("contexts", [])

    prediction = run.outputs.get("answer", "") # Assuming final answer is in 'answer' key

    llm = ChatOpenAI(model="gpt-4o", temperature=0)
    answer_grader = grade_prompt_hallucinations | llm

    try:
        score_response = answer_grader.invoke({
            "documents": contexts,
            "student_answer": prediction
        })
        raw_score_text = score_response.content.strip().lower() # Access content attribute for ChatMessage
        score = 1.0 if raw_score_text.startswith("yes") else 0.0 # Check if it starts with 'yes'
    except Exception as e:
        logging.error(f"Error grading hallucination: {e}", exc_info=True)
        score = 0.0

    print(f'question="{input_question}", hallucination_score={score}')
    return {"key": "answer_hallucination", "score": score}


def docs_relevance_evaluator(run, example) -> dict:
    """
    Evaluates if the retrieved documents are relevant to the input question.
    """
    input_question = example.inputs.get("question", "")
    # Similar assumption as above: 'contexts' is in the agent run output
    # Correcting to access contexts from the custom prediction function's output
    contexts = run.outputs.get("contexts", [])

    llm = ChatOpenAI(model="gpt-4o", temperature=0)
    answer_grader = grade_prompt_doc_relevance | llm

    try:
        score_response = answer_grader.invoke({
            "question": input_question,
            "documents": contexts
        })
        raw_score_text = score_response.content.strip().lower() # Access content attribute for ChatMessage
        score = 1.0 if raw_score_text.startswith("yes") else 0.0 # Check if it starts with 'yes'
    except Exception as e:
        logging.error(f"Error grading document relevance: {e}", exc_info=True)
        score = 0.0

    print(f'question="{input_question}", doc_relevance_score={score}')
    return {"key": "document_relevance", "score": score}


# --- Async Prediction Function ---
async def predict_agent_run_for_eval(inputs: Dict[str, Any], agent_instance, retriever_instance) -> Dict[str, Any]:
    """
    Uses provided agent and retriever instances to answer a question and provide retrieved contexts.
    Takes inputs dictionary directly from example.inputs.
    """
    question = inputs.get("question")
    if not question:
        logging.error("Question key missing from inputs dictionary.")
        return {"answer": "Error: Question missing.", "contexts": []}

    logging.info(f"Evaluating question with agent: {question}")

    # Retrieve documents asynchronously
    retrieved_docs_strings = []
    try:
        if retriever_instance is None:
             raise ValueError("Retriever instance is None.")
        # Ensure retriever_instance is a runnable with ainvoke
        if not hasattr(retriever_instance, 'ainvoke'):
             logging.error("Retriever instance does not have an 'ainvoke' method.")
             retrieved_docs = []
        else:
            retrieved_docs = await retriever_instance.ainvoke(question)
        retrieved_docs_strings = [doc.page_content for doc in retrieved_docs]
    except Exception as e: # Catch broader exceptions during retrieval
        logging.warning(f"Failed to retrieve documents for evaluation: {e}")


    # Generate answer asynchronously using the provided agent instance
    try:
        if agent_instance is None:
             raise ValueError("Agent instance is None.")
        # Ensure agent_instance is a runnable with ainvoke
        if not hasattr(agent_instance, 'ainvoke'):
             logging.error("Agent instance does not have an 'ainvoke' method.")
             final_answer = "Error: Agent instance does not have 'ainvoke'."
        else:
            # Invoke the agent with the question
            # The agent expects {"input": question}
            response = await agent_instance.ainvoke({"input": question})
            final_answer = response.get("output", "No answer generated.") # Assuming final answer is in 'output' key

        # Return both the answer and the separately retrieved contexts for evaluators
        return {"answer": final_answer, "contexts": retrieved_docs_strings}

    except Exception as e: # Catch broader exceptions during agent execution
        logging.error(f"Error during agent execution for evaluation: {e}", exc_info=True)
        return {"answer": f"Error during agent execution: {e}", "contexts": retrieved_docs_strings}


# Wrap the async prediction function in a RunnableLambda
# This runnable will be the target for aevaluate
runnable_predict = RunnableLambda(lambda inputs: predict_agent_run_for_eval(inputs, agent, retriever))


# --- Run Evaluation ---
dataset_name = DATASET_NAME  # Ensure this matches the dataset created above

async def main():
    # Ensure the 'dataset' object is accessible in this scope
    if 'dataset' not in globals() or dataset is None:
         raise ValueError("Dataset object not found or is None. Ensure it's created in a previous cell.")
    # Ensure 'agent' and 'retriever' objects are accessible
    if 'agent' not in globals() or agent is None:
        raise ValueError("Agent object not found or is None. Ensure it's initialized.")
    if 'retriever' not in globals() or retriever is None:
        raise ValueError("Retriever object not found or is None. Ensure it's initialized.")


    experiment_results = await aevaluate(
        # Pass the RunnableLambda as the target
        runnable_predict,
        data=dataset.id, # Pass the dataset ID for evaluation
        evaluators=[
            answer_evaluator,
            answer_hallucination_evaluator,
            docs_relevance_evaluator
        ],
        experiment_prefix=f"RAG_AI_Content_Eval_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
        metadata={
            "variant": "Evaluation with specific questions from LangSmith dataset (RunnableLambda target)",
            "source_dataset_name": dataset_name
        },
    )

    print("\n--- Evaluation complete! ---")
    try:
        print(f"View results on LangSmith: {experiment_results.url}")
    except AttributeError:
        print(f"Experiment name: {experiment_results.experiment_name}")


# Directly await main() since Colab has a running event loop
await main()

In [ ]:
import asyncio
from langsmith import Client
import os
from datetime import datetime
from langchain.agents import initialize_agent, AgentType # Assuming these are needed for the factory
from langchain_openai import ChatOpenAI # Assuming this is needed for the factory
from langchain.memory import ConversationBufferMemory # Assuming this is needed for the factory
from langchain.tools import Tool # Assuming this is needed for the factory
from typing import Dict, Any # Import Dict and Any for type hinting

# Assuming 'client', 'dataset', 'tools', 'llm', and 'memory' objects are available from previous cells

# --- Define Agent Factory ---
def create_agent():
    """
    Factory function that initializes and returns the agent.
    Assumes 'tools', 'llm', and 'memory' are available in the global scope.
    """
    # Ensure required components are available
    if 'tools' not in globals():
        raise ValueError("Global 'tools' list not found.")
    if 'llm' not in globals():
         raise ValueError("Global 'llm' object not found.")
    # The memory object is created within the factory, so no need to check global scope

    # Re-initialize the agent within the factory
    # Note: Memory might behave differently when initialized inside a factory
    # for each run. Consider how you want memory to be handled across examples.
    # For simple evaluation on independent examples, a new memory per run is fine.
    agent_instance = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
        memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True), # Initialize memory here if needed per run
        verbose=True,
        handle_parsing_errors=True
    )
    return agent_instance

# --- Input Mapper for Dataset ---
def map_inputs(example_inputs: Dict[str, Any]) -> Dict[str, Any]:
    """Maps dataset example inputs ('question') to the agent's expected input format ('input')."""
    # The agent expects 'input' for the current query and 'chat_history'.
    # run_on_dataset handles the 'chat_history' for conversational agents.
    # We just need to map the dataset's 'question' key to the agent's 'input' key.
    return {"input": example_inputs.get("question", "")}


# --- Run Agent on Dataset ---
async def run_evaluation_dataset():
    """
    Runs the agent on the specified dataset and logs results to LangSmith.
    """
    if 'client' not in globals() or client is None:
         raise ValueError("LangSmith client not found or is None. Ensure it's initialized.")
    if 'dataset' not in globals() or dataset is None:
         raise ValueError("Dataset object not found or is None. Ensure it's created in a previous cell.")

    print(f"Running agent on dataset '{dataset.name}'...")

    # Use client.run_on_dataset with the factory function and input_mapper
    results = await client.run_on_dataset(
        dataset_name=dataset.name,
        llm_or_chain_factory=create_agent, # Pass the factory function
        input_mapper=map_inputs, # Pass the input mapper function
        experiment_prefix=f"RunOnDataset_AI_Content_Eval_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
        project_name=os.environ.get("LANGCHAIN_PROJECT", "default"), # Use the project name from environment
        concurrency=1, # Start with concurrency 1 to avoid potential memory issues with ConversationBufferMemory per run
        verbose=True,
        # This argument might also be required depending on langsmith version/setup
        # project_kwargs={} # Add any project specific kwargs here if needed
    )

    print("\n--- Evaluation run complete! ---")
    try:
        # Print the URL to view the experiment results in LangSmith
        # The structure of 'results' might vary slightly based on langsmith version
        # Attempt to find a URL in the response
        if hasattr(results, 'url'):
             print(f"View results on LangSmith: {results.url}")
        elif isinstance(results, list) and len(results) > 0 and hasattr(results[0], 'url'):
             # Fallback for older versions or different structures
             print(f"View results on LangSmith (first run URL): {results[0].url.rsplit('/', 1)[0]}") # Get experiment URL from run URL
        else:
             print("Results logged to LangSmith. Check your project dashboard.")
    except Exception as e:
        print(f"Could not generate LangSmith URL: {e}")
        print("Results logged to LangSmith. Check your project dashboard.")

# Run the async function
await run_evaluation_dataset()

After the execution completes, click the link in the output to view the traces in LangSmith. You can then manually inspect each run to see the agent's steps, the retrieved documents, and the final answer, and compare them to the expected output in the dataset.

In [ ]:
# --- Imports ---
import logging
import os
import asyncio
from typing import List, Dict, Any
from datetime import datetime

from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.documents import Document
from langsmith.evaluation import aevaluate

# --- Import chatbot backend components ---
try:
    from chatbot_backend import agent, retriever
    logging.info("✅ Successfully imported agent and retriever")
except ImportError as e:
    logging.error(f"❌ Failed to import from chatbot_backend.py: {e}")
    raise SystemExit("Exiting: Could not load core chatbot components.")

# --- Ensure API key is set ---
if "OPENAI_API_KEY" not in os.environ or not os.environ["OPENAI_API_KEY"]:
    raise ValueError("❌ OPENAI_API_KEY not set. Please load it into environment variables.")

# --- Async helper to retrieve documents ---
async def get_relevant_documents(query: str) -> List[Document]:
    if retriever is None:
        raise ValueError("Retriever is None. Ensure chatbot_backend.py initializes it correctly.")
    return await retriever.ainvoke(query)

# --- Load evaluation prompts from LangChain Hub ---
grade_prompt_hallucinations = hub.pull("langchain-ai/rag-answer-hallucination")
grade_prompt_doc_relevance = hub.pull("langchain-ai/rag-document-relevance")
grade_prompt_answer = hub.pull("langchain-ai/rag-answer-vs-reference")

# --- Evaluator functions here (same as your version) ---
# answer_evaluator(...)
# answer_hallucination_evaluator(...)
# docs_relevance_evaluator(...)

# --- Prediction function ---
async def predict_agent_run_for_eval(example: Dict[str, Any]) -> Dict[str, Any]:
    question = example.get("question")
    if not question:
        logging.error("Question key missing from example inputs.")
        return {"answer": "Error: Question missing.", "contexts": []}

    logging.info(f"Evaluating question: {question}")

    retrieved_docs_strings = []
    try:
        retrieved_docs = await get_relevant_documents(question)
        retrieved_docs_strings = [doc.page_content for doc in retrieved_docs]
    except Exception as e:
        logging.warning(f"Failed to retrieve documents: {e}")

    try:
        response = await agent.ainvoke({"input": question})
        final_answer = response.get("output", "No answer generated.")
        return {"answer": final_answer, "contexts": retrieved_docs_strings}
    except Exception as e:
        logging.error(f"Error during agent execution: {e}", exc_info=True)
        return {"answer": f"Error during agent execution: {e}", "contexts": retrieved_docs_strings}

# --- Run Evaluation ---
dataset_name = "rag_eval_dataset"  # 👈 update with your dataset name in LangSmith

async def main():
    experiment_results = await aevaluate(
        predict_agent_run_for_eval,
        data=dataset_name,
        evaluators=[
            answer_evaluator,
            answer_hallucination_evaluator,
            docs_relevance_evaluator,
        ],
        experiment_prefix=f"RAG_AI_Content_Eval_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
        metadata={"variant": "Evaluation with specific questions", "source_dataset_name": dataset_name},
    )

    print("\n--- Evaluation complete! ---")
    try:
        print(f"View results on LangSmith: {experiment_results.url}")
    except AttributeError:
        print(f"Experiment name: {experiment_results.experiment_name}")

# 👇 In Jupyter/Colab, call this in a separate cell:
# await main()


 ================ BACK UP ==================

In [ ]:
# # --- Module Docstring ---
# """
# This script creates a question-answer dataset for everything related to AI content creation in LangSmith.
# It fetches the API key from Colab secrets, sets the environment variables,
# defines a set of specific Q&A pairs, and uploads them to LangSmith.
# """

# # --- Standard library imports ---
# import logging
# import os
# from typing import List, Dict

# # --- Colab secret handling ---
# from google.colab import userdata

# # --- Third-party imports ---
# from langsmith import Client

# # --- Configure logging ---
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# # --- Load LangSmith API key from Colab secrets ---
# langsmith_api_key = userdata.get("LANGCHAIN_API_KEY")

# if not langsmith_api_key:
#     raise ValueError("❌ No LangSmith API key found in Colab secrets!")

# # Set environment variables for LangSmith
# os.environ["LANGCHAIN_API_KEY"] = langsmith_api_key
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

# logging.info("✅ LangSmith API key loaded and environment configured.")

# # --- Initialize LangSmith Client ---
# client = Client()

# # Define dataset name
# DATASET_NAME = "rag_eval_dataset"

# # --- Define Q&A pairs ---
# qa_pairs: List[Dict[str, Dict[str, str]]] = [
#     {
#         "inputs": {
#             "question": (
#                 "I’m starting as an AI content creator in 2025. Can you list the most essential AI tools I should use, and explain what each one is best for?"
#             )
#         },
#         "outputs": {
#             "answer": (
#                 "As a beginner AI content creator in 2025, here are some must-have tools and their strengths:\n"
#                 "- ChatGPT (OpenAI): Best for idea generation, drafting scripts, and long-form blog posts.\n"
#                 "- Jasper: Focused on marketing copy — great for ad campaigns, landing pages, and social posts.\n"
#                 "- MidJourney: Specialized in generating high-quality AI images, thumbnails, and visual assets.\n"
#                 "- Runway: A leader in AI video editing and generative video production.\n"
#                 "- Grammarly AI: Ensures writing is clear, polished, and grammatically correct.\n"
#                 "- Notion AI: Enhances productivity by organizing ideas, outlines, and workflows."
#             )
#         }
#     },
#     {
#         "inputs": {
#             "question": (
#                 "I write long blog articles. How can AI help me repurpose them into short-form content for social media platforms like Instagram, TikTok, and LinkedIn?"
#             )
#         },
#         "outputs": {
#             "answer": (
#                 "AI can transform a single blog post into multiple content formats:\n"
#                 "- Copy.ai / Jasper: Summarize sections into captions, taglines, or tweet threads.\n"
#                 "- ChatGPT: Reframe insights into engaging Q&A carousels or LinkedIn posts.\n"
#                 "- Canva AI: Turn text into visually appealing infographics or slides.\n"
#                 "- MidJourney: Create supporting visuals (e.g., diagrams, story slides).\n"
#                 "- Pictory AI: Convert blogs into video scripts and short-form content for TikTok/Reels."
#             )
#         }
#     },
#     {
#         "inputs": {
#             "question": (
#                 "I want to make video content using AI. Which tools should I use for scriptwriting, video generation, and editing, and how do they fit together in a workflow?"
#             )
#         },
#         "outputs": {
#             "answer": (
#                 "For a complete AI-powered video workflow in 2025:\n"
#                 "1. Scriptwriting → ChatGPT or Jasper for scripts, hooks, and CTAs.\n"
#                 "2. Video Generation → Runway Gen-3 for cinematic AI video; Pika Labs for short animations.\n"
#                 "3. Avatar & Narration → Synthesia for talking avatars, ElevenLabs for natural voiceovers.\n"
#                 "4. Editing → Descript to edit videos by text, auto-generate subtitles, and polish content."
#             )
#         }
#     },
#     {
#         "inputs": {
#             "question": (
#                 "How can AI improve the SEO of my blog posts and ensure they rank higher in Google searches? Can you name specific tools and how they help?"
#             )
#         },
#         "outputs": {
#             "answer": (
#                 "AI-powered SEO tools to boost rankings:\n"
#                 "- SurferSEO: Recommends keywords, structure, and length based on competitors.\n"
#                 "- Clearscope: Uses NLP to include semantically related terms.\n"
#                 "- SEMRush AI Assistant: Tracks keyword rankings and suggests updates.\n"
#                 "- MarketMuse: Identifies content gaps for untapped opportunities.\n"
#                 "- ChatGPT (SEO plugins): Drafts keyword-rich meta descriptions and FAQ sections."
#             )
#         }
#     },
#     {
#         "inputs": {
#             "question": (
#                 "I often struggle to come up with new content ideas. How can AI assist me in brainstorming creative content topics, campaign ideas, or trending formats for 2025?"
#             )
#         },
#         "outputs": {
#             "answer": (
#                 "AI can generate fresh, trend-driven ideas:\n"
#                 "- ChatGPT: Brainstorms article titles, podcast themes, and outlines.\n"
#                 "- Jasper: Creates ad campaign slogans and A/B test variants.\n"
#                 "- Copy.ai: Produces multiple versions of headlines and hooks.\n"
#                 "- BuzzSumo + AI: Finds trending topics, viral posts, and competitor insights.\n"
#                 "- Notion AI: Organizes brainstormed ideas into campaigns, schedules, and task lists."
#             )
#         }
#     }
# ]

In [ ]:
# # Check if dataset exists or create it
# dataset = client.create_dataset(dataset_name="RAG_eval_dataset", description="RAG evaluation dataset for AI content creation")
# logging.info(f"✅ Dataset '{dataset.name}' is ready in LangSmith.")

In [ ]:
# # --- Create or load dataset ---
# try:
#     dataset = client.read_dataset(dataset_name=DATASET_NAME)
#     logging.info("📂 Dataset '%s' already exists with ID: %s", DATASET_NAME, dataset.id)
# except Exception as err:
#     logging.info("📂 Dataset '%s' not found. Creating a new one... (Error: %s)", DATASET_NAME, err)
#     dataset = client.create_dataset(
#         dataset_name=DATASET_NAME,
#         description="Dataset for evaluating AI content creation tools and strategies."
#     )
#     # Add examples
#     for qa_pair in qa_pairs:
#         client.create_example(inputs=qa_pair["inputs"], outputs=qa_pair["outputs"], dataset_id=dataset.id)
#     logging.info("✅ Added %d examples to dataset '%s'.", len(qa_pairs), DATASET_NAME)

In [ ]:
# from langchain.callbacks import LangChainTracer
# from langchain.agents import initialize_agent, AgentType

# tracer = LangChainTracer()
# agent_with_tracing = initialize_agent(
#     tools=tools,
#     llm=llm,
#     agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, #ZERO_SHOT_REACT_DESCRIPTION
#     memory=memory,
#     verbose=True,
#     handle_parsing_errors=True,
#     callbacks=[tracer]   # 👈 now every run is logged to LangSmith
# )